In [ ]:
!pip install datasets

In [ ]:
!pip install transformers

In [ ]:
!pip install accelerate

In [ ]:
!pip install fuzzywuzzy

In [ ]:
!pip install python-Levenshtein

In [ ]:
import random
from typing import Dict, List, Union
from datasets import load_dataset
from fuzzywuzzy import fuzz, process
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from transformers import pipeline
def prepare_message_for_llm(text: Union[str, List[str]], categories: Dict[str, str]) -> Dict[str, Union[List[Dict[str, str]], List[List[Dict[str, str]]]]]:
    if len(categories) < 2:
        raise RuntimeError(f'The category list is too small! Expected 2 or more categories, got {len(categories)} ones.')
    categories_ = sorted(list(categories.keys()))
    categories_as_string = ', '.join(categories_[:-1]) + ' и ' + categories_[-1]
    if isinstance(text, str):
        prompt = f'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного ' \
                 f'списка тем наиболее представлена в следующем тексте. ' \
                 f'В качестве ответа напишите только название темы из списка, больше ничего.\n' \
                 f'Список тем: {categories_as_string}.\n'
        for cur in categories_:
            prompt += f'Текст: {" ".join(categories[cur].split())}\nВаш ответ: {cur}\n'
        prompt += f'Текст: {" ".join(text.split())}\nВаш ответ: '
        messages = [
            {
                'role': 'system',
                'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.'
            },
            {
                'role': 'user',
                'content': prompt
            }
        ]
    else:
        messages = []
        for it in text:
            prompt = f'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного ' \
                     f'списка тем наиболее представлена в следующем тексте. ' \
                     f'В качестве ответа напишите только название темы из списка, больше ничего.\n' \
                     f'Список тем: {categories_as_string}.\n'
            for cur in categories_:
                prompt += f'Текст: {" ".join(categories[cur].split())}\nВаш ответ: {cur}\n'
            prompt += f'Текст: {" ".join(text.split())}\nВаш ответ: '
            messages.append([
                {
                    'role': 'system',
                    'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.'
                },
                {
                    'role': 'user',
                    'content': prompt
                }
            ])
    return {'message_for_llm': messages}
llm_pipeline = pipeline(model='Qwen/Qwen2-7B-Instruct', device_map='auto', torch_dtype='auto')
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))
print(f'Categories for classification are: {list_of_categories}')
print(validation_set)
examples_by_categories = dict()
for current_category in list_of_categories:
    examples_by_categories[current_category] = random.choice(
        train_set.filter(lambda it: it['category'] == current_category)['text']
    )
    print(f'Category: {current_category}\n')
    print(f'Random text: {examples_by_categories[current_category]}\n\n')
validation_set_for_llm = validation_set.map(lambda it: prepare_message_for_llm(it['text'], examples_by_categories))
test_set_for_llm = test_set.map(lambda it: prepare_message_for_llm(it['text'], examples_by_categories))
print(validation_set_for_llm)
print(validation_set['text'][0])
print(validation_set_for_llm['message_for_llm'][0])
y_pred = list(map(
    lambda x: llm_pipeline(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(validation_set_for_llm['message_for_llm'])
))
y_true = validation_set['category']
print(y_pred[0])
print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))
y_pred_with_normalization = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    y_pred
))
print(y_pred_with_normalization[0])
print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))
y_pred = list(map(
    lambda x: llm_pipeline(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(test_set_for_llm['message_for_llm'])
))
y_true = test_set['category']
print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))
y_pred_with_normalization = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    y_pred
))
print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))

In [ ]:
print(validation_set_for_llm['message_for_llm'][0])

[{'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.', 'role': 'system'}, {'content': 'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного списка тем наиболее представлена в следующем тексте. В качестве ответа напишите только название темы из списка, больше ничего.\nСписок тем: entertainment, geography, health, politics, science/technology, sports и travel.\nТекст: Созерцание цветения сакуры, называемое "ханами", вошло в японскую культуру еще в VIII веке.\nВаш ответ: entertainment\nТекст: Спутники, каждый из которых был тяжелее 1 000 фунтов и перемещался со скоростью приблизительно 17 500 миль в час, столкнулись на высоте 491 мили над поверхностью Земли.\nВаш ответ: geography\nТекст: Прочие варианты, которые основаны на биологических ритмах, включают в себя прием жидкости в больших количествах (в частности, воды или чая, известного мочегонного средства) перед сном, что заставляет человека вставать, ч

In [ ]:
y_pred = list(map(
    lambda x: llm_pipeline(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(validation_set_for_llm['message_for_llm'])
))
y_true = validation_set['category']

  0%|          | 0/99 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
print(y_pred[0])

[{'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.', 'role': 'system'}, {'content': 'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного списка тем наиболее представлена в следующем тексте. В качестве ответа напишите только название темы из списка, больше ничего.\nСписок тем: entertainment, geography, health, politics, science/technology, sports и travel.\nТекст: Созерцание цветения сакуры, называемое "ханами", вошло в японскую культуру еще в VIII веке.\nВаш ответ: entertainment\nТекст: Спутники, каждый из которых был тяжелее 1 000 фунтов и перемещался со скоростью приблизительно 17 500 миль в час, столкнулись на высоте 491 мили над поверхностью Земли.\nВаш ответ: geography\nТекст: Прочие варианты, которые основаны на биологических ритмах, включают в себя прием жидкости в больших количествах (в частности, воды или чая, известного мочегонного средства) перед сном, что заставляет человека вставать, ч

In [ ]:
print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))

                    precision    recall  f1-score   support

           culture       0.00      0.00      0.00         0
     entertainment       0.67      0.67      0.67         9
         geography       0.78      0.88      0.82         8
            health       0.82      0.82      0.82        11
          politics       0.93      0.93      0.93        14
science/technology       0.91      0.84      0.87        25
          security       0.00      0.00      0.00         0
            sports       0.92      1.00      0.96        12
           traffic       0.00      0.00      0.00         0
            travel       0.88      0.70      0.78        20
          wildlife       0.00      0.00      0.00         0

          accuracy                           0.83        99
         macro avg       0.54      0.53      0.53        99
      weighted avg       0.86      0.83      0.84        99



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_pred_with_normalization = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    y_pred
))

In [ ]:
print(y_pred_with_normalization[0])

sports


In [ ]:
print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))

                    precision    recall  f1-score   support

     entertainment       0.67      0.67      0.67         9
         geography       0.78      0.88      0.82         8
            health       0.82      0.82      0.82        11
          politics       0.87      0.93      0.90        14
science/technology       0.91      0.84      0.87        25
            sports       0.86      1.00      0.92        12
            travel       0.83      0.75      0.79        20

          accuracy                           0.84        99
         macro avg       0.82      0.84      0.83        99
      weighted avg       0.84      0.84      0.84        99



In [ ]:
y_pred = list(map(
    lambda x: llm_pipeline(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(test_set_for_llm['message_for_llm'])
))
y_true = test_set['category']

  0%|          | 0/204 [00:00<?, ?it/s]

In [ ]:
print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))

                    precision    recall  f1-score   support

           animals       0.00      0.00      0.00         0
               art       0.00      0.00      0.00         0
    communications       0.00      0.00      0.00         0
     entertainment       0.87      0.68      0.76        19
         geography       0.74      0.82      0.78        17
            health       0.83      0.91      0.87        22
       immigration       0.00      0.00      0.00         0
        literature       0.00      0.00      0.00         0
             media       0.00      0.00      0.00         0
             music       0.00      0.00      0.00         0
          politics       0.94      0.97      0.95        30
          religion       0.00      0.00      0.00         0
science/technology       0.90      0.86      0.88        51
            sports       0.88      0.84      0.86        25
        technology       0.00      0.00      0.00         0
         transport       0.00      0.00

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_pred_with_normalization = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    y_pred
))

In [ ]:
print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))

                    precision    recall  f1-score   support

     entertainment       0.76      0.68      0.72        19
         geography       0.74      0.82      0.78        17
            health       0.71      0.91      0.80        22
          politics       0.88      0.97      0.92        30
science/technology       0.90      0.88      0.89        51
            sports       0.81      0.84      0.82        25
            travel       0.94      0.72      0.82        40

          accuracy                           0.84       204
         macro avg       0.82      0.83      0.82       204
      weighted avg       0.85      0.84      0.84       204

